In [5]:
import os
import numpy as np
import pandas as pd
import re

from keras.models import load_model
from keras.preprocessing import image
from keras.applications import Xception

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
test_folder = "./test2/"
feature_size = 299

model_path = "./Xception_whole.h5"
model = load_model(model_path)

#middle_faeture_shape = (10,10,2048)
#X_model = Xception(weights=None,include_top=False,input_shape=(feature_size,feature_size,3))
#X_model.load_weights("./xception_weights_notop.h5")

print(model.summary())

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 10, 10, 2048)      20861480  
_________________________________________________________________
flatten_1 (Flatten)          (None, 204800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 650)               133120650 
_________________________________________________________________
dropout_1 (Dropout)          (None, 650)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 12)                7812      
Total params: 153,989,942
Trainable params: 153,935,414

In [3]:
def preprocessing(x,output_size = None,pre_model = None):
    if pre_model is None:
        return x
    elif output_size is None:
        raise Exception('should use output_size for pre_model')
    else:
        middle_faeture = pre_model.predict(x)
        res = np.reshape(middle_faeture,(1,output_size[0] * output_size[1] * output_size[2]))
        return res

In [6]:
res = []

for f in os.listdir(test_folder):
    img = image.load_img(test_folder+f,target_size=(feature_size,feature_size))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x/255
    x = preprocessing(x,None,None)
    pre = model.predict(x)
    label_num = np.argmax(pre[0])
    if label_num == 0:
        label = 'norm'
    else:
        label = 'defect'+str(label_num)
    res.append({
        'in':int(f.split('.')[0]),
        'file':f,
        'label':label
    })

In [7]:
tmp = pd.DataFrame(res)
tmp = tmp.sort_values('in')
df = tmp.drop('in',1)

print(tmp.head())
print(df.head())

      file  in    label
681  0.jpg   0  defect7
144  1.jpg   1  defect2
399  2.jpg   2     norm
981  3.jpg   3     norm
42   4.jpg   4     norm
      file    label
681  0.jpg  defect7
144  1.jpg  defect2
399  2.jpg     norm
981  3.jpg     norm
42   4.jpg     norm


In [8]:
df.to_csv('submit.csv',header=False,index=False)